# Чат-бот

- Команды (несколько)

Например: 
+ /start -> ответить сообщением "Привет!"
+ /help -> сказать какие функции может бот выполнять        
---

### Домашнее задание
Почитать про строки:
- https://pythonworld.ru/tipy-dannyx-v-python/stroki-funkcii-i-metody-strok.html

Допилить бота:
- Добавить валидации
- Добавить шестой пункт: выбор Airline (к нему тоже валидацию нужно добавить)
- Свои сценарии придумать (*)
- Добавить /help

### Разработка бота

In [23]:
def handle(message):
    message = message.lower()

    if message == '/start':
        print('Привет!')
    elif message == '/help':
        print('Напишите /start, чтобы начать')
    elif 'привет' in message:
        print('И вам привет!')
    else:
        print('Извините, я вас не понял')

In [89]:
import datetime
import joblib

import pandas as pd

from haversine import haversine

In [90]:
# Загрузка модели

pipe_lnr = joblib.load('pipe_lnr.joblib')

Функциональные требования:
- stage0: /find_ticket
- stage1: Спрашивает дату вылета
- stage2: Спрашивает город отправления/город приземления (нужно выбрать из какого списка) 'Якутск' -> 'Такого города в базе нет, выбирайте из: 'Delhi, Mumbai, Bangalore,...'
- stage3: Спрашивает эконом или бизнес
- stage4: За сколько хочет добраться
- stage5: Во сколько хочет вылететь


distance - вычислить

speed - distance / taken_time_minutes

week_day - брать из даты вылета

stop_count - (если скорость < 100 -> 2 остановки, < 350 -> 1 остановка, в противном случае 0)

airline - подобрать самый дешевый vs самый дорогой

In [96]:
import json
import requests
import urllib


TOKEN = "5109235392:AAHa9Glr_PO_zKpk9C45CDl-Qexp7uMlixo"
URL = "https://api.telegram.org/bot{}/".format(TOKEN)
USERNAME_BOT = "test"


def get_url(url):
    response = requests.get(url)
    content = response.content.decode("utf8")
    return content


def get_json_from_url(url):
    content = get_url(url)
    js = json.loads(content)
    return js


def get_updates(offset=None):
    url = URL + "getUpdates?timeout=100"
    if offset:
        url += "&offset={}".format(offset)
    js = get_json_from_url(url)
    return js


def get_last_update_id(updates):
    update_ids = []
    for update in updates["result"]:
        update_ids.append(int(update["update_id"]))
    return max(update_ids)


def send_message(text, chat_id):
    tot = urllib.parse.quote_plus(text)
    url = URL + "sendMessage?text={}&chat_id={}".format(tot, chat_id)
    get_url(url)


def send_document(doc, chat_id):
    files = {'document': open(doc, 'rb')}
    requests.post(URL + "sendDocument?chat_id={}".format(chat_id), files=files)


def send_image(doc, chat_id):
    files = {'photo': open(doc, 'rb')}
    requests.post(URL + "sendPhoto?chat_id={}".format(chat_id), files=files)

    
# - stage0: /find_ticket
# - stage1: Спрашивает дату вылета
# - stage2: Спрашивает город отправления/город приземления (нужно выбрать из какого списка) 'Якутск' -> 'Такого города в базе нет, выбирайте из: 'Delhi, Mumbai, Bangalore,...'
# - stage3: Спрашивает эконом или бизнес
# - stage4: За сколько хочет добраться
# - stage5: Во сколько хочет вылететь
# - stage6: Пусть выберет Airline



chats_stage = {}   # chat_id: stage_number
parameters = {} # chat_id: {1: '04.04.2001', 2: 'channai, dumbai', 3: 'эконом', 4: '3 часа', 5: '10:00'}
CITIES_COORD = {
    'Delhi': [77.21667, 28.66667],
    'Mumbai': [72.83333, 18.96667],
    'Bangalore': [77.59796, 12.96991],
    'Kolkata': [88.33778, 22.54111],
    'Hyderabad': [78.46667, 17.36667],
    'Chennai': [80.27, 13.09]
}

def process_stage(stage_number, chat_id, message=None):
    if message is not None:
        if chat_id not in parameters:
            parameters[chat_id] = {}
    
        parameters[chat_id][stage_number] = message
    
    if stage_number == 0:
        send_message('Когда вы собираетесь полететь? Например: 20.04.2022', chat_id)
    
    if stage_number == 1:
        # валидация
        try:
            date = datetime.datetime.strptime(message, '%d.%m.%Y')
            
            if date < datetime.datetime.now():
                send_message('Введите дату не ранее сегодняшнего', chat_id)
                chats_stage[chat_id] = stage_number - 2
            else:
                send_message('Введите город отправления и прибытия через запятую. Например "Chennai, Dumpai"', chat_id)
        except:
            send_message('Введите, пожалуйста, корректную дату. Например: 20.04.2022', chat_id)
            chats_stage[chat_id] = stage_number - 2
                    
    if stage_number == 2:
        # валидацию сделать так, чтобы первый и второй город, записанные через запятую входили в хэш-таблицу CITIES_COORD
        send_message('Вы продпочитаете эконом или бизнес?', chat_id)
        
    if stage_number == 3:
        # валидация
        
        if message.lower() not in ['эконом', 'бизнес']:
            send_message('Напишите пожалуйста "эконом" или "бизнес"', chat_id)
            chats_stage[chat_id] = stage_number - 2
        else:
            send_message('За сколько часов вы хотите добраться?', chat_id)
        
    if stage_number == 4:
        # валидация
        send_message('Во сколько вы хотите вылететь?', chat_id)
    
    if stage_number == 5:
        dep_date = parameters[chat_id][1]  # 20.04.2022
        dep_date = datetime.datetime.strptime(dep_date, '%d.%m.%Y')
        
        from_to_cities = parameters[chat_id][2]  # Chennai, Dumbai
        econom_or_business = parameters[chat_id][3]  # эконом
        taken_time = parameters[chat_id][4]  # '3'
        dep_time = parameters[chat_id][5]  # 10:00
        
        from_city, to_city = from_to_cities.split(',')
        from_city_coord = CITIES_COORD[from_city.strip()]
        to_city_coord = CITIES_COORD[to_city.strip()]
        
        is_econom = 0
        
        if econom_or_business.lower() == 'эконом':
            is_econom = 1
            
        taken_time_minutes = int(taken_time) * 60
        dep_minutes = int(dep_time.split(':')[0]) * 60 + int(dep_time.split(':')[1])
        week_day = dep_date.weekday()
        
        
        distance = haversine(from_city_coord, to_city_coord)
        speed = distance / (taken_time_minutes / 60)
        
        stop_count = 0
        
        if speed < 100:
            stop_count = 2
        elif speed < 350:
            stop_count = 1
            
        airlines = ['Vistara', 'Air India', 'Other', 'Indigo']
        prices = []
        
        for airline in airlines:
            row = pd.DataFrame({
                'is_econom': [is_econom],
                'taken_time_minues': [taken_time_minutes],
                'distance': [distance],
                'speed': [speed],
                'week_day': [week_day],
                'dep_minutes': [dep_minutes],
                'stop_count': [stop_count],
                'new_airlines': [airline],
            })
            
            price = pipe_lnr.predict(row)[0]
            prices.append(price)
        
        min_price_airline = airlines[prices.index(min(prices))]
        max_price_airline = airlines[prices.index(max(prices))]
        
        send_message(f'Полет компанией {min_price_airline} будет стоить {round(min(prices))}', chat_id)
        send_message(f'Полет компанией {max_price_airline} будет стоить {round(max(prices))}', chat_id)
        
        return True
    
    return False
        

def echo_all(updates):
    for update in updates["result"]:
        if update.get("message") != None:
            if update.get("message", {}).get("text") != None:
                message = update['message']['text']  # 04.04.2001
                chat_id = update['message']['chat']['id']  # 45885
                
                if message == '/find_ticket':
                    process_stage(0, chat_id)
                    chats_stage[chat_id] = 0
                elif chat_id in chats_stage:
                    prev_stage = chats_stage[chat_id]
                    is_finished = process_stage(prev_stage + 1, chat_id, message)
                    
                    if is_finished:
                        del chats_stage[chat_id]
                    else:
                        chats_stage[chat_id] += 1
                else:
                    send_message('Напишите /find_ticket, чтобы найти билет', chat_id)
                

def main():
    last_update_id = None
    while True:
        updates = get_updates(last_update_id)
        if updates is not None:
            if len(updates["result"]) > 0:
                last_update_id = get_last_update_id(updates) + 1
                echo_all(updates)

In [97]:
main()

KeyboardInterrupt: 

In [44]:
real_columns = ['is_econom', 'taken_time_minues', 'distance', 'speed', 'week_day', 'dep_minutes', 'stop_count']
cat_columns = ['new_airlines']